In [ ]:
!nvidia-smi

# GD

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
#@title another1
!sudo add-apt-repository -y ppa:alessandro-strada/ppa 2>&1 > /dev/null
!sudo apt-get update -qq 2>&1 > /dev/null
!sudo apt -y install -qq google-drive-ocamlfuse 2>&1 > /dev/null
!google-drive-ocamlfuse

In [ ]:
#@title another2
!sudo apt-get install -qq w3m # to act as web browser 
!xdg-settings set default-web-browser w3m.desktop # to set default browser
%cd /content
!mkdir drive
%cd drive
!mkdir MyDrive
%cd ..
%cd ..
!google-drive-ocamlfuse /content/drive/MyDrive

# train set

In [ ]:
#@title Training
Model = "SAEHD" #@param ["AMP","SAEHD", "Quick96", "XSeg"]
Backup_every_hour = True #@param {type:"boolean"}
Silent_Start = True #@param {type:"boolean"}

%cd "/content/workspace"

#Mount Google Drive as folder
from google.colab import drive
drive.mount('/content/drive')

import psutil, os, time

p = psutil.Process(os.getpid())
uptime = time.time() - p.create_time()

if (Backup_every_hour):
  print("Time to end session: "+str(round((86400-uptime)/3600))+" hours")
  backup_time = str(14120)
  backup_cmd = " --execute-program -"+backup_time+" \"import os; os.system('zip -r -q /content/model.zip model'); os.system('cp /content/model.zip /content/drive/My\ Drive/'); print('Backed up!') \"" 
elif (round(82080-uptime) > 0):
  print("Time to backup: "+str(round((82080-uptime)/3600))+" hours")
  backup_time = str(round(82080-uptime))
  backup_cmd = " --execute-program "+backup_time+" \"import os; os.system('zip -r -q /content/model.zip model'); os.system('cp /content/model.zip /content/drive/My\ Drive/'); print('Backed up!') \"" 
else:
  print("Session expires in less than an hour.")
  backup_cmd = ""
    
cmd = "/content/bonanzi/main.py train --training-data-src-dir /content/workspace/data_src/aligned --training-data-dst-dir /content/workspace/data_dst/aligned --pretraining-data-dir /content/pretrain --model-dir /content/workspace/model --model "+Model

if Model == "Quick96":
  cmd+= " --pretrained-model-dir pretrain_Q96"

if Silent_Start:
  cmd+= " --silent-start"

if (backup_cmd != ""):
  train_cmd = (cmd+backup_cmd)
else:
  train_cmd = (cmd)

!python $train_cmd

In [ ]:
#@title Install custom
%cd /content
Mode = "install" #@param ["install", "update"]

from pathlib import Path
if (Mode == "install"):
  !git clone https://github.com/goodluckmasa/bonanzi.git

  # fix linux warning
  # /usr/lib/python3.6/multiprocessing/semaphore_tracker.py:143: UserWarning: semaphore_tracker: There appear to be 1 leaked semaphores to clean up at shutdown
  fin = open("/usr/lib/python3.6/multiprocessing/semaphore_tracker.py", "rt")
  data = fin.read()
  data = data.replace('if cache:', 'if False:')
  fin.close()

  fin = open("/usr/lib/python3.6/multiprocessing/semaphore_tracker.py", "wt")
  fin.write(data)
  fin.close()
else:
  %cd /content/bonanzi
  !git pull

!pip uninstall -y tensorflow
!pip install -r /content/bonanzi/requirements-colab.txt
!pip install --upgrade scikit-image
!apt-get install cuda-10-0
!pip install numpy --upgrade --ignore-installed

if not Path("/content/workspace").exists():
  !mkdir /content/workspace; mkdir /content/workspace/data_src; mkdir /content/workspace/data_src/aligned; mkdir /content/workspace/data_dst; mkdir /content/workspace/data_dst/aligned; mkdir /content/workspace/model  

import IPython
from google.colab import output

display(IPython.display.Javascript('''
 function ClickConnect(){
   btn = document.querySelector("colab-connect-button")
   if (btn != null){
     console.log("Click colab-connect-button"); 
     btn.click() 
     }
   
   btn = document.getElementById('ok')
   if (btn != null){
     console.log("Click reconnect"); 
     btn.click() 
     }
  }
  
setInterval(ClickConnect,60000)
'''))

print("\nDone!")

In [ ]:
#@title change from yaw to pitch

from pathlib import Path

src_file = Path('/content/bonanzi/samplelib/SampleGeneratorFace.py')
dst_file = Path('/content/bonanzi/samplelib/alpha.tmp')

with src_file.open() as fin, \
     dst_file.open(mode='x') as fout:

     fout.write(
         fin.read().replace('s_yaw = -pyr[1]', 's_yaw = -pyr[0]')
     )


dst_file.replace(
    src_file
)


In [ ]:
#@title Import Model

from google.colab import drive
drive.mount('/content/drive', force_remount=True)

!cp /content/drive/MyDrive/model.zip /content
!unzip -q /content/model.zip -d /content/workspace

In [ ]:
!cp /content/drive/MyDrive/data_src/faceset.pak /content/workspace/data_src/aligned

!cp /content/drive/MyDrive/data_dst/faceset.pak /content/workspace/data_dst/aligned

In [ ]:
#@title get FFHQ pretrain from github
!wget -q --no-check-certificate -r 'https://github.com/goodluckmasa/colab/releases/download/test/faceset.zip' -O /content/faceset.zip
!mkdir /content/pretrain
!unzip -q /content/faceset.zip -d /content/pretrain/
!rm /content/faceset.zip


In [ ]:
#@title power pretrain
!mkdir /content/pretrain
!cp /content/drive/MyDrive/pretrain/faceset.pak /content/pretrain/faceset.pak

# another

In [ ]:
#@title Import Model


!cp /content/drive/MyDrive/model.zip /content
!unzip -q /content/model.zip -d /content/workspace

In [ ]:
!cp /content/drive/MyDrive/data_src/faceset.pak /content/workspace/data_src/aligned/faceset.pak

!cp /content/drive/MyDrive/data_dst/faceset.pak /content/workspace/data_dst/aligned/faceset.pak

In [ ]:
#@title change from yaw to pitch

from pathlib import Path

src_file = Path('/content/bonanzi/samplelib/SampleGeneratorFace.py')
dst_file = Path('/content/bonanzi/samplelib/alpha.tmp')

with src_file.open() as fin, \
     dst_file.open(mode='x') as fout:

     fout.write(
         fin.read().replace('s_yaw = -pyr[1]', 's_yaw = -pyr[0]')
     )


dst_file.replace(
    src_file
)


In [ ]:
#@title get FFHQ pretrain from github
!wget -q --no-check-certificate -r 'https://github.com/goodluckmasa/colab/releases/download/test/faceset.zip' -O /content/faceset.zip
!mkdir /content/pretrain
!unzip -q /content/faceset.zip -d /content/pretrain/
!rm /content/faceset.zip


# extract prepare

In [ ]:
#@title Install
%cd /content
Mode = "install" #@param ["install", "update"]

from pathlib import Path
if (Mode == "install"):
  !git clone -b test https://github.com/goodluckmasa/bonanzi.git

  # fix linux warning
  # /usr/lib/python3.6/multiprocessing/semaphore_tracker.py:143: UserWarning: semaphore_tracker: There appear to be 1 leaked semaphores to clean up at shutdown
  fin = open("/usr/lib/python3.6/multiprocessing/semaphore_tracker.py", "rt")
  data = fin.read()
  data = data.replace('if cache:', 'if False:')
  fin.close()

  fin = open("/usr/lib/python3.6/multiprocessing/semaphore_tracker.py", "wt")
  fin.write(data)
  fin.close()
else:
  %cd /content/bonanzi
  !git pull

!pip uninstall -y tensorflow
!pip install -r /content/bonanzi/requirements-colab.txt
!pip install --upgrade scikit-image
!apt-get install cuda-10-0
!pip install numpy --upgrade --ignore-installed

if not Path("/content/workspace").exists():
  !mkdir /content/workspace; mkdir /content/workspace/data_src; mkdir /content/workspace/data_src/aligned; mkdir /content/workspace/data_dst; mkdir /content/workspace/data_dst/aligned; mkdir /content/workspace/model  

import IPython
from google.colab import output

display(IPython.display.Javascript('''
 function ClickConnect(){
   btn = document.querySelector("colab-connect-button")
   if (btn != null){
     console.log("Click colab-connect-button"); 
     btn.click() 
     }
   
   btn = document.getElementById('ok')
   if (btn != null){
     console.log("Click reconnect"); 
     btn.click() 
     }
  }
  
setInterval(ClickConnect,60000)
'''))

print("\nDone!")

In [ ]:
#@title extrract 1080P

from pathlib import Path

src_file = Path('/content/bonanzi/facelib/S3FDExtractor.py')
dst_file = Path('/content/bonanzi/facelib/alpha.tmp')

with src_file.open() as fin, \
     dst_file.open(mode='x') as fout:

     fout.write(
         fin.read().replace('if min(r-l,bt) < 40: #filtering faces < 40pix by any side', 'if min(r-l,bt) < 340: #filtering faces < 40pix by any side')
     )


dst_file.replace(
    src_file
)

In [ ]:
#@title extrract 2160P

from pathlib import Path

src_file = Path('/content/bonanzi/facelib/S3FDExtractor.py')
dst_file = Path('/content/bonanzi/facelib/alpha.tmp')

with src_file.open() as fin, \
     dst_file.open(mode='x') as fout:

     fout.write(
         fin.read().replace('if min(r-l,bt) < 40: #filtering faces < 40pix by any side', 'if min(r-l,bt) < 600: #filtering faces < 600pix by any side')
     )


dst_file.replace(
    src_file
)

In [ ]:
#@title extrract dst

from pathlib import Path

src_file = Path('/content/bonanzi/facelib/S3FDExtractor.py')
dst_file = Path('/content/bonanzi/facelib/alpha.tmp')

with src_file.open() as fin, \
     dst_file.open(mode='x') as fout:

     fout.write(
         fin.read().replace('if min(r-l,bt) < 40: #filtering faces < 40pix by any side', 'if min(r-l,bt) < 180: #filtering faces < 600pix by any side')
     )


dst_file.replace(
    src_file
)

In [ ]:
#@title install gpen

!git clone https://github.com/Cioscos/GPEN-Cioscos.git
!mkdir /content/input
!mkdir /content/weights
!wget -q --no-check-certificate -r 'https://github.com/goodluckmasa/colab/releases/download/gpen/GPEN-BFR-256-D.pth' -O /content/weights/GPEN-BFR-256-D.pth
!wget -q --no-check-certificate -r 'https://github.com/goodluckmasa/colab/releases/download/gpen/GPEN-BFR-256.pth' -O /content/weights/GPEN-BFR-256.pth
!wget -q --no-check-certificate -r 'https://github.com/goodluckmasa/colab/releases/download/gpen/GPEN-BFR-512-D.pth' -O /content/weights/GGPEN-BFR-512-D.pth
!wget -q --no-check-certificate -r 'https://github.com/goodluckmasa/colab/releases/download/gpen/GPEN-BFR-512.pth' -O /content//weights/GPEN-BFR-512.pth
!wget -q --no-check-certificate -r 'https://github.com/goodluckmasa/colab/releases/download/gpen/GPEN-Colorization-1024.pth' -O /content/weights/GPEN-Colorization-1024.pth
!wget -q --no-check-certificate -r 'https://github.com/goodluckmasa/colab/releases/download/gpen/GGPEN-Inpainting-1024.pth' -O /content/weights/GPEN-Inpainting-1024.pth
!wget -q --no-check-certificate -r 'https://github.com/goodluckmasa/colab/releases/download/gpen/GPEN-Seg2face-512.pth' -O /content/weights/GPEN-Seg2face-512.pth
!wget -q --no-check-certificate -r 'https://github.com/goodluckmasa/colab/releases/download/gpen/model_ir_se50.pth' -O /content//weights/model_ir_se50.pth
!wget -q --no-check-certificate -r 'https://github.com/goodluckmasa/colab/releases/download/gpen/ParseNet-latest.pth' -O /content/weights/ParseNet-latest.pth
!wget -q --no-check-certificate -r 'https://github.com/goodluckmasa/colab/releases/download/gpen/RetinaFace-R50.pth' -O /content//weights/RetinaFace-R50.pth
!wget -q --no-check-certificate -r 'https://github.com/goodluckmasa/colab/releases/download/gpen/rrdb_realesrnet_psnr.pth' -O /content/weights/rrdb_realesrnet_psnr.pth
!echo -e "opencv-python\nnumpy\nscikit-image\npillow\nninja\ntorch\ntorchvision\nopencv-python\nscipy\ntqdm" >> /content/requirement.txt
!pip3 install -r /content/requirement.txt



In [ ]:
#@title modify py

from pathlib import Path

src_file = Path('/content/GPEN-Cioscos/face_enhancement.py')
dst_file = Path('/content/GPEN-Cioscos/alpha.py')

with src_file.open() as fin, \
     dst_file.open(mode='x') as fout:

     fout.write(
         fin.read().replace('import __download_weights', '')
     )


dst_file.replace(
    src_file
)

# extract dst

In [ ]:
import shutil
shutil.copy('','/content/workspace/data_src.mp4')

!python /content/bonanzi/main.py videoed extract-video --input-file /content/workspace/data_src.* --fps 0 --output-ext jpg --output-dir /content/workspace/data_src/

In [ ]:
!python /content/bonanzi/main.py extract --max-faces-from-image 2 --image-size 640 --jpeg-quality 100 --input-dir /content/workspace/data_src --output-dir /content/workspace/data_src/aligned --force-gpu-idxs 0 --face-type whole_face --detector s3fd --no-output-debug

!python /content/bonanzi/main.py util --input-dir /content/workspace/data_src/aligned  --pack-faceset

!install -D /content/workspace/data_src/aligned/faceset.pak /content/drive/MyDrive/work//faceset.pak

!rm -r /content/workspace/data_src

# faceset

In [ ]:
import shutil
shutil.copy('','/content/workspace/data_src.mp4')

!python /content/bonanzi/main.py videoed extract-video --input-file /content/workspace/data_src.* --fps 12 --output-ext jpg --output-dir /content/workspace/data_src/

In [ ]:
!rm -r /content/workspace/data_src.mp4
import os
import glob
path = "/content/workspace/data_src"
files = glob.glob(path + '/*.jpg')
for f in files:
    os.rename(f, os.path.join(path, 'las' + os.path.basename(f)))

In [ ]:
!python /content/bonanzi/main.py extract --max-faces-from-image 2 --image-size 640 --jpeg-quality 100 --input-dir /content/workspace/data_src --output-dir /content/workspace/data_src/aligned --force-gpu-idxs 0 --face-type whole_face --detector s3fd --no-output-debug

!python /content/bonanzi/main.py util --input-dir /content/workspace/data_src/aligned  --pack-faceset

!install -D /content/workspace/data_src/aligned/faceset.pak /content/drive/MyDrive/work//faceset.pak

!rm -r /content/workspace/data_src

# faceset

In [ ]:
import shutil
shutil.copy('','/content/workspace/data_src.mp4')

!python /content/bonanzi/main.py videoed extract-video --input-file /content/workspace/data_src.* --fps 12 --output-ext jpg --output-dir /content/workspace/data_src/

In [ ]:
!rm -r /content/workspace/data_src.mp4
import os
import glob
path = "/content/workspace/data_src"
files = glob.glob(path + '/*.jpg')
for f in files:
    os.rename(f, os.path.join(path, 'las' + os.path.basename(f)))

In [ ]:
!python /content/bonanzi/main.py extract --max-faces-from-image 2 --image-size 640 --jpeg-quality 100 --input-dir /content/workspace/data_src --output-dir /content/workspace/data_src/aligned --force-gpu-idxs 0 --face-type whole_face --detector s3fd --no-output-debug

!python /content/bonanzi/main.py util --input-dir /content/workspace/data_src/aligned  --pack-faceset

!install -D /content/workspace/data_src/aligned/faceset.pak /content/drive/MyDrive/work//faceset.pak

!rm -r /content/workspace/data_src

# rife


In [ ]:
!nvidia-smi

In [ ]:
!git clone https://github.com/hzwer/arXiv2020-RIFE

In [ ]:
from google.colab import drive
drive._mount('/content/drive')

In [ ]:
#@title model
!wget -q --no-check-certificate -r 'https://github.com/goodluckmasa/colab/releases/download/rife/RIFE_trained_model_v3.6.zip' -O /content/RIFE_trained_model_v3.6.zip
!7z e RIFE_trained_model_v3.6.zip

In [ ]:
#@title prepare
!mkdir /content/arXiv2020-RIFE/train_log
!mv *.pkl *.py /content/arXiv2020-RIFE/train_log/
%cd /content/arXiv2020-RIFE/
!gdown --id 1i3xlKb7ax7Y70khcTcuePi6E7crO_dFc
!pip3 install -r requirements.txt

In [ ]:
!python3 inference_video.py --exp=1 --video=

# Manage workspace






In [ ]:
#@title Import Model

from google.colab import drive
drive.mount('/content/drive', force_remount=True)

!cp /content/drive/MyDrive/model.zip /content
!unzip -q /content/model.zip -d /content/workspace

In [ ]:
#@title Delete and recreate
Mode = "Delete and recreate workspace" #@param ["Delete and recreate workspace", "Delete models", "Delete data_src", "Delete data_src aligned", "Delete data_src video", "Delete data_dst", "Delete data_dst aligned", "Delete merged frames"]

%cd "/content" 

if Mode == "Delete and recreate workspace":
  cmd = "rm -r /content/workspace ; mkdir /content/workspace; mkdir /content/workspace/data_src; mkdir /content/workspace/data_src/aligned; mkdir /content/workspace/data_dst; mkdir /content/workspace/data_dst/aligned; mkdir /content/workspace/model"  
elif Mode == "Delete models":
  cmd = "rm -r /content/workspace/model/*"
elif Mode == "Delete data_src":
  cmd = "rm /content/workspace/data_src/*.png || rm -r /content/workspace/data_src/*.jpg"
elif Mode == "Delete data_src aligned":
  cmd = "rm -r /content/workspace/data_src/aligned/*"
elif Mode == "Delete data_src video":
  cmd = "rm -r /content/workspace/data_src.*"
elif Mode == "Delete data_dst":
  cmd = "rm /content/workspace/data_dst/*.png || rm /content/workspace/data_dst/*.jpg"
elif Mode == "Delete data_dst aligned":
  cmd = "rm -r /content/workspace/data_dst/aligned/*"
elif Mode == "Delete merged frames":
  cmd = "rm -r /content/workspace/data_dst/merged; rm -r /content/workspace/data_dst/merged_mask"
  
!$cmd
print("Done!")

# dfd

In [ ]:
#@title Prepare Colab machine
!pip install dominate

from IPython.display import clear_output
from google.colab import files
from PIL import Image
import matplotlib.pyplot as plt
import warnings
import os

warnings.filterwarnings("ignore")
%matplotlib inline

!rm -rf /content/sample_data
!git clone --branch size https://github.com/goodluckmasa/DFDNet.git

os.chdir('/content/DFDNet/')
!pip install -r requirements.txt

link_prefix = 'https://github.com/rocketsvm/DFDNet/releases/download/20200827/'
wget_options = '-q --show-progress --progress=bar:noscroll --no-check-certificate -r'
!wget $wget_options $link_prefix'latest_net_G.pth' -O 'latest_net_G.pth'
!wget $wget_options $link_prefix'left_eye_32_center.npy' -O 'left_eye_32_center.npy'
!wget $wget_options $link_prefix'left_eye_64_center.npy' -O 'left_eye_64_center.npy'
!wget $wget_options $link_prefix'left_eye_128_center.npy' -O 'left_eye_128_center.npy'
!wget $wget_options $link_prefix'left_eye_256_center.npy' -O 'left_eye_256_center.npy'
!wget $wget_options $link_prefix'right_eye_32_center.npy' -O 'right_eye_32_center.npy'
!wget $wget_options $link_prefix'right_eye_64_center.npy' -O 'right_eye_64_center.npy'
!wget $wget_options $link_prefix'right_eye_128_center.npy' -O 'right_eye_128_center.npy'
!wget $wget_options $link_prefix'right_eye_256_center.npy' -O 'right_eye_256_center.npy'
!wget $wget_options $link_prefix'mouth_32_center.npy' -O 'mouth_32_center.npy'
!wget $wget_options $link_prefix'mouth_64_center.npy' -O 'mouth_64_center.npy'
!wget $wget_options $link_prefix'mouth_128_center.npy' -O 'mouth_128_center.npy'
!wget $wget_options $link_prefix'mouth_256_center.npy' -O 'mouth_256_center.npy'
!wget $wget_options $link_prefix'nose_32_center.npy' -O 'nose_32_center.npy'
!wget $wget_options $link_prefix'nose_64_center.npy' -O 'nose_64_center.npy'
!wget $wget_options $link_prefix'nose_128_center.npy' -O 'nose_128_center.npy'
!wget $wget_options $link_prefix'nose_256_center.npy' -O 'nose_256_center.npy'
!wget $wget_options $link_prefix'vgg19.pth' -O 'vgg19.pth'
!mkdir -p /content/DFDNet/checkpoints/facefh_dictionary
!mv latest_net_G.pth /content/DFDNet/checkpoints/facefh_dictionary
!mkdir -p /content/DFDNet/DictionaryCenter512
!mv *_center.npy /content/DFDNet/DictionaryCenter512
!mkdir -p /content/DFDNet/weights
!mv vgg19.pth /content/DFDNet/weights

!nvidia-smi -L

# print("0%/100%   Copying has started and will take about 4 minutes.")
# !rsync -ah --progress '/content/drive/My Drive/DFDNet/checkpoints/facefh_dictionary/latest_net_G.pth' '/content/DFDNet/checkpoints/facefh_dictionary'
# print("33%/100%  Checkpoints copied")
# !rsync -ah --progress  '/content/drive/My Drive/DFDNet/weights/vgg19.pth' '/content/DFDNet/weights/'
# print("66%/100%  Weights copied")
# !rsync -ah --progress -r '/content/drive/My Drive/DFDNet/DictionaryCenter512/' '/content/DFDNet/DictionaryCenter512/'
# print("100%/100% Dictionary copied")

In [ ]:
#@title move aligned photos
!cp -r /content/workspace/data_src/aligned /content/DFDNet/aligned
!rm -r /content/workspace/data_src

In [ ]:
!rm -r '/content/DFDNet/aligned'
!rm -r '/content/DFDNet/aligned_dfdn'
!mkdir '/content/DFDNet/aligned'
!mkdir '/content/DFDNet/aligned_dfdn'

#pack

In [ ]:
#@title packing
Type = "gpen" #@param ["srcdrive", "dstdrive","srccontet","dstcontent","dfd","gpen"]
Download = "false" #@param ["false", "true"]

if Type == "srcdrive":
  %cd "/content"
  !python bonanzi/main.py util --input-dir "/content/drive/MyDrive/1purlin/barn/workspace/data_src/aligned/"  --pack-faceset

if Type == "dstdrive":
  %cd "/content"
  !python bonanzi/main.py util --input-dir "/content/drive/MyDrive/1purlin/barn/workspace/data_dst/aligned/"  --pack-faceset

if Type == "srccontet":
  %cd "/content"
  !python bonanzi/main.py util --input-dir "workspace/data_src/aligned/"  --pack-faceset

if Type == "dstcontent":
  %cd "/content"
  !python bonanzi/main.py util --input-dir "workspace/data_dst/aligned/"  --pack-faceset

if Type == "dfd":
  %cd "/content"
  !python bonanzi/main.py util --input-dir "DFDNet/aligned_dfdn/"  --pack-faceset

if Type == "gpen":
  %cd "/content"
  !python bonanzi/main.py util --input-dir "/content/results"  --pack-faceset

In [ ]:
#@title move to drive
%cd "/content/results"
!install -D faceset.pak /content/drive/MyDrive/work//faceset.pak
%cd

In [ ]:
#@title del
!rm -r '/content/results'
!rm -r '/content/input'
!mkdir '/content/results'
!mkdir '/content/input'

#merge

In [ ]:
#@title Merge
Model = "SAEHD" #@param ["SAEHD", "Quick96" ]

cmd = "bonanzi/main.py merge --input-dir workspace/data_dst --output-dir workspace/data_dst/merged --output-mask-dir workspace/data_dst/merged_mask --aligned-dir workspace/data_dst/aligned --model-dir workspace/model --model "+Model

%cd "/content"
!python $cmd

In [ ]:
#@title Get result video 
Mode = "result video" #@param ["result video", "result_mask video"]
Copy_to_Drive = True #@param {type:"boolean"}


if Mode == "result video":
  !python bonanzi/main.py videoed video-from-sequence --input-dir workspace/data_dst/merged --output-file workspace/result.mp4 --reference-file workspace/data_dst.mp4 --include-audio
  if Copy_to_Drive:
    !cp /content/workspace/result.mp4 /content/drive/My\ Drive/
elif Mode == "result_mask video":
  !python bonanzi/main.py videoed video-from-sequence --input-dir workspace/data_dst/merged_mask --output-file workspace/result_mask.mp4 --reference-file workspace/data_dst.mp4
  if Copy_to_Drive:
    !cp /content/workspace/result_mask.mp4 /content/drive/My\ Drive/


# add extract set

In [ ]:
#@title inmport vid
import shutil
shutil.copy('','/content/workspace/data_src.mp4')

In [ ]:
#@title Extract frames
Video = "data_src" #@param ["data_src", "data_dst"]

%cd "/content"

cmd = "bonanzi/main.py videoed extract-video "

if Video == "data_dst":
  cmd+= " --input-file workspace/data_dst.* --fps 0 --output-ext jpg --output-dir workspace/data_dst/"
else:
  cmd+= " --input-file workspace/data_src.* --fps 12 --output-ext jpg --output-dir workspace/data_src/"
  
!python $cmd

In [ ]:
#@title change frame name
import os
import glob
path = "/content/workspace/data_src"
files = glob.glob(path + '/*.jpg')
for f in files:
    os.rename(f, os.path.join(path, 'las' + os.path.basename(f)))
!rm -r /content/workspace/data_src.mp4

In [ ]:
#@title Detect faces
Data = "data_src" #@param ["data_src", "data_dst"]
Detector = "S3FD (whole face)" #@param ["S3FD", "S3FD (whole face)"]
Debug = False #@param {type:"boolean"}

detect_type = "s3fd"
dbg = " --output-debug" if Debug else " --no-output-debug"

folder = "workspace/"+Data
folder_aligned = folder+"/aligned"

cmd = "bonanzi/main.py extract --max-faces-from-image 2 --image-size 640 --jpeg-quality 100 --input-dir "+folder+" --output-dir "+folder_aligned
cmd+=" --detector "+detect_type+" --force-gpu-idxs 0"+dbg

if "whole face" in Detector:
  cmd+=" --face-type whole_face" 
%cd "/content"
!python $cmd

In [ ]:
#@title aligned to gpen
!cp -r /content/workspace/data_src/aligned /content/input
!rm -r /content/workspace/data_src

In [ ]:
#@title run gpen

!python /content/GPEN-Cioscos/face_enhancement.py --model GPEN-BFR-512 --use_cuda --indir /content/input --outdir /content/results


In [ ]:
#@title packing
Type = "gpen" #@param ["srcdrive", "dstdrive","srccontet","dstcontent","dfd","gpen"]
Download = "false" #@param ["false", "true"]

if Type == "srcdrive":
  %cd "/content"
  !python bonanzi/main.py util --input-dir "/content/drive/MyDrive/1purlin/barn/workspace/data_src/aligned/"  --pack-faceset

if Type == "dstdrive":
  %cd "/content"
  !python bonanzi/main.py util --input-dir "/content/drive/MyDrive/1purlin/barn/workspace/data_dst/aligned/"  --pack-faceset

if Type == "srccontet":
  %cd "/content"
  !python bonanzi/main.py util --input-dir "workspace/data_src/aligned/"  --pack-faceset

if Type == "dstcontent":
  %cd "/content"
  !python bonanzi/main.py util --input-dir "workspace/data_dst/aligned/"  --pack-faceset

if Type == "dfd":
  %cd "/content"
  !python bonanzi/main.py util --input-dir "DFDNet/aligned_dfdn/"  --pack-faceset

if Type == "gpen":
  %cd "/content"
  !python bonanzi/main.py util --input-dir "/content/results"  --pack-faceset

In [ ]:
#@title move to drive
%cd "/content/results"
!install -D faceset.pak /content/drive/MyDrive/work//faceset.pak
%cd

In [ ]:
#@title del
!rm -r '/content/results'
!rm -r '/content/input'
!mkdir '/content/results'
!mkdir '/content/input'

In [ ]:
!python /content/bonanzi/main.py extract --max-faces-from-image 2 --image-size 640 --jpeg-quality 100 --input-dir /content/workspace/data_src --output-dir /content/workspace/data_src/aligned --force-gpu-idxs 0 --face-type whole_face --detector s3fd --no-output-debug

!cp -r /content/workspace/data_src/aligned /content/input
!rm -r /content/workspace/data_src

!python /content/GPEN-Cioscos/face_enhancement.py --model GPEN-BFR-512 --use_cuda --indir /content/input --outdir /content/results

!python /content/bonanzi/main.py util --input-dir /content/results  --pack-faceset

!install -D /content/results/faceset.pak /content/drive/MyDrive/work//faceset.pak

!rm -r '/content/results'
!rm -r '/content/input'
!mkdir '/content/results'
!mkdir '/content/input'